In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
import random
import numpy as np

# Разбиение обучающих данных
# labels_num -  количество меток (от 0 до 9)
# train_len - длина того обучающего множества, которое мы хотим использовать для обучения, оно должно делиться на количество меток
def balancing(full_train_dataset, lables_num, train_len):
  # Длина всего обучающего множества и того обучающего множества, которое мы хотим использовать для обучения
  full_train_len = len(full_train_dataset)
  # Количество данных с одной меткой
  label_group_num = int(train_len/lables_num)

  # Создаём группы для хранения индексов каждой метки в обучающем наборе данных
  label_groups_index = [[] for _ in range(lables_num)]
  for i in range(full_train_len):
    label = full_train_dataset[i][1]
    label_groups_index[label].append(i)

  # Обрезаем группы, оставляя случайные, неповторяющиеся элементы в каждой и объединяем их всех в один набор индексов
  all_index = np.array([], dtype=int)
  for i in range(lables_num):
    all_index = np.append(all_index, random.sample(label_groups_index[i], label_group_num))
  np.random.shuffle(all_index)

  # Формируем обучающий набор данных
  train_dataset = torch.utils.data.Subset(full_train_dataset, all_index)
  return train_dataset

In [ ]:
# Определение модели
# neurons_num - количество нейронов в каждом слое
# img_size - размер изображений из MNIST
class Net(nn.Module):
    def __init__(self, neurons_num, img_size):
        super(Net, self).__init__()
        self.img_size = img_size
        self.fc1 = nn.Linear(img_size, neurons_num)
        self.fc2 = nn.Linear(neurons_num, neurons_num)
        self.fc3 = nn.Linear(neurons_num, neurons_num)
        self.fc4 = nn.Linear(neurons_num, neurons_num)
        self.fc5 = nn.Linear(neurons_num, neurons_num)
        self.fc6 = nn.Linear(neurons_num, 10)

    def forward(self, x):
        x = x.view(-1, self.img_size)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = nn.functional.relu(self.fc3(x))
        x = nn.functional.relu(self.fc4(x))
        x = nn.functional.relu(self.fc5(x))
        x = self.fc6(x)
        return x

In [ ]:
import time
# Обучение модели
def train(model, train_dataset, test_dataset, batch_size, learning_rate, epochs):
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    # Инициализация загрузчиков данных
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    loss_array = []
    accuracy_array = []

    startTime = time.time()
    # Обучение модели
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(images)
            loss = nn.functional.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()

        # Проверка точности модели на тестовом наборе данных
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        loss_array.append(loss.item())
        accuracy_array.append(accuracy)
        print('{:.2f} Epoch [{}/{}], Loss: {:.6f}, Accuracy: {:.2f}%'.format(time.time() - startTime, epoch+1, epochs, loss.item(), accuracy))

In [ ]:
# Загрузка данных MNIST
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

In [ ]:
train_dataset = balancing(train_dataset, 10, 30000)

In [ ]:
# Инициализация модели
model = Net(1000, 28*28)
train(model, train_dataset, test_dataset, 64, 0.1, 5)

In [ ]:
for epoch in range(epochs):
  print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, epochs, loss_array[epoch], accuracy_array[epoch]))